State Laws Generation:

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import os
import uuid
import time
import json
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import os
import re
import json
import requests
from bs4 import BeautifulSoup
import PyPDF2

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

chrome_options.binary_location = '/usr/bin/google-chrome-stable'  # Path to your Chrome binary

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [ ]:
def scrape_and_download_pdf_and_titles(url, button_section_id, section_name, max_num, municipality=None):
    if (municipality):
        base_path = f"state/{section_name}/{municipality}"
    else:
        base_path = f"state/{section_name}"

    if not os.path.exists(f"state/{section_name}"):
        if (municipality):
            os.mkdir(f"state/{section_name}")

        os.mkdir(base_path)
        os.mkdir(f"{base_path}/pdfs")

    schema_list = []

    driver.get(url)

    driver_button_section = driver.find_element(By.ID, button_section_id)
    driver_button_section.click()
    time.sleep(1)

    for i in range(max_num):
        dropdown = Select(driver.find_element(By.ID, "ddlPagina"))
        time.sleep(1)
        dropdown.select_by_visible_text(f"{i+1} de {max_num}")
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table', {'id': 'dgData'})
        rows = table.find_all('tr')

        for ind, row in enumerate(rows):
            if (ind != 0):
                document_name = row.find('td', {'class': 'tabla_i_documento'})
                if (document_name != None):
                    document_name = document_name.text
                    dates = row.find_all('td', {'class': 'tabla_i_fecha'})
                    last_reform_date = dates[1].text
                    try:
                        pdf_link = row.find('td', {'class': 'tabla_pdfs'}).find_all('a', href=True)[1]['href']
                    except:
                        pdf_link = row.find('td', {'class': 'tabla_pdfs'}).find_all('a', href=True)[0]['href']
                    path_id = re.search(r'/([^/]*)\.pdf', pdf_link).group(1) if re.search(r'/([^/]*)\.pdf', pdf_link) else None

                    if not "Ingresos" in document_name and not "Egresos" in document_name:
                        schema_list.append({
                            "title": document_name,
                            "path_id": path_id,
                            "source": pdf_link,
                            "last_reform_date": last_reform_date
                        })

    with open(f"{base_path}/schema.json", 'w') as f:
        json.dump(schema_list, f, indent=4)


    for file in schema_list:
        r = requests.get(file["source"])

        pdf_file_path = f"{base_path}/pdfs/{file['path_id']}/{file['path_id']}.pdf"

        if not os.path.exists(pdf_file_path):

            os.mkdir(f"{base_path}/pdfs/{file['path_id']}")

            with open(pdf_file_path, "wb") as pdf:
                pdf.write(r.content)
            print(f"✅File processed: {file['source']}")
        else:
            print(f"❌File already exists: {file['source']}")


In [ ]:
url = "http://sistec.nl.gob.mx/Transparencia_2015_LyPOE/Acciones/Legislacion.aspx"
button_section_id = "btnLeyesEstatales"
section_name = "leyes_estatales"
max_num = 9
scrape_and_download_pdf_and_titles(url, button_section_id, section_name, max_num)

In [ ]:
url = "http://sistec.nl.gob.mx/Transparencia_2015_LyPOE/Acciones/Legislacion.aspx"
button_section_id = "btnReglamentosEstatales"
section_name = "reglamentos_estatales"
max_num = 8
scrape_and_download_pdf_and_titles(url, button_section_id, section_name, max_num)

In [ ]:
url = "http://sistec.nl.gob.mx/Transparencia_2015_LyPOE/Acciones/Legislacion.aspx"
button_section_id = "btnReglamentosMunicipales"
section_name = "reglamentos_municipales"
municipality = "monterrey"
max_num = 3
scrape_and_download_pdf_and_titles(url, button_section_id, section_name, max_num, municipality)

In [ ]:
elements_folder = os.listdir("state/reglamentos_municipales/monterrey/pdfs")
print(len(elements_folder))

Federal Laws Generation:
Download zip from https://www.diputados.gob.mx/LeyesBiblio/legiscd.htm

Actual Laws:

In [ ]:
base_path = "federal/pdfs"
pdfs = os.listdir(base_path)

schema_list = []

for pdf in pdfs:
    pdf_name = pdf[:-4]
    path_id = str(uuid.uuid4())
    mod_time = os.path.getmtime(f"{base_path}/{pdf}")
    formatted_time = time.ctime(mod_time)

    schema_list.append({
        "title": pdf_name,
        "path_id": path_id,
        "source": "https://www.diputados.gob.mx/LeyesBiblio/legiscd.htm",
        "last_reform_date": formatted_time
    })
    os.mkdir(f"{base_path}/{path_id}")
    os.rename(f"{base_path}/{pdf}", f"{base_path}/{path_id}/{path_id}.pdf")
    print(f"✅File Processed: {pdf_name}")

with open(f"federal/schema.json", 'w') as f:
    json.dump(schema_list, f, indent=4)

Regulations for the laws:

In [ ]:
url = "https://www.diputados.gob.mx/LeyesBiblio/regla.htm"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

table = soup.find('table', {'id': 'table5'})
rows = table.find_all('tr')

schema_list = []

for ind, row in enumerate(rows):
    if (ind != 0):
        col1 = row.find_all('td')[1]
        document_name = col1.find("p").text.replace("\n", "")

        col2 = row.find_all('td')[3]
        pdf_link = col2.find("a", href=True)["href"]
        
        path_id = str(uuid.uuid4())

        schema_list.append({
            "title": document_name,
            "path_id": path_id,
            "source": f"https://www.diputados.gob.mx/LeyesBiblio/{pdf_link}",
            "last_reform_date": ""
        })

base_path = "federal/reglamentos_federales"

for file in schema_list:
    r = requests.get(file["source"])

    pdf_file_path = f"{base_path}/pdfs/{file['path_id']}/{file['path_id']}.pdf"

    if not os.path.exists(pdf_file_path):

        os.mkdir(f"{base_path}/pdfs/{file['path_id']}")

        with open(pdf_file_path, "wb") as pdf:
            pdf.write(r.content)
        print(f"✅File processed: {file['source']}")
    else:
        print(f"❌File already exists: {file['source']}")

with open(f"federal/reglamentos_federales/schema.json", 'w') as f:
   json.dump(schema_list, f, indent=4)